# The Tail Hedge Debate: AQR vs Universa

## The Question

Can buying deep out-of-the-money puts **improve** long-term portfolio returns, or is it always a drag?

## The Two Camps

### Universa / Spitznagel (2021)

A small allocation $w$ to deep OTM puts enhances geometric compounding:

$$G = \mathbb{E}[\ln(1 + R_p)] \quad \text{where} \quad R_p = (1-w) \cdot R_{\text{SPY}} + w \cdot R_{\text{puts}}$$

Claim: with $w \approx 3.3\%$, the portfolio CAGR rises from ~10% to ~12.3% because crash protection allows you to stay fully invested.

### AQR / Ilmanen & Israelov (2018)

The cost of tail hedging systematically exceeds the benefit:

$$\mathbb{E}[\text{Put P\&L}] < 0 \quad \text{because} \quad \sigma_{\text{implied}} > \sigma_{\text{realized}}$$

Diversification across asset classes is cheaper than buying puts. Simply **reducing equity exposure** achieves the same drawdown reduction without the premium bleed.

## Our Test

We test 4 put-buying configurations against SPY, from standard OTM to deep OTM (Universa-style).

In [ ]:
import os, sys, warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

PROJECT_ROOT = os.path.realpath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, PROJECT_ROOT)
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'notebooks'))
os.chdir(PROJECT_ROOT)

from backtest_runner import (
    load_data, run_backtest, INITIAL_CAPITAL,
    make_puts_strategy, make_deep_otm_put_strategy,
)
from nb_style import apply_style, shade_crashes, color_excess, style_returns_table, FT_GREEN, FT_RED

apply_style()
%matplotlib inline
print('Ready.')

In [ ]:
data = load_data()
schema = data['schema']
spy_prices = data['spy_prices']

---
## Run: Standard Puts vs Deep OTM Puts at Various Allocations

In [ ]:
configs = [
    # Standard OTM puts (delta -0.25 to -0.10)
    ('OTM Puts 0.2%',   0.998, 0.002, lambda: make_puts_strategy(schema)),
    ('OTM Puts 1.0%',   0.99,  0.01,  lambda: make_puts_strategy(schema)),
    ('OTM Puts 2.0%',   0.98,  0.02,  lambda: make_puts_strategy(schema)),
    # Deep OTM puts (delta -0.10 to -0.02) — Universa-style
    ('Deep OTM 0.1%',   0.999, 0.001, lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 0.3%',   0.997, 0.003, lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 1.0%',   0.99,  0.01,  lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 3.3%',   0.967, 0.033, lambda: make_deep_otm_put_strategy(schema)),
]

results = []
for name, s_pct, o_pct, fn in configs:
    print(f'  {name}...', end=' ', flush=True)
    r = run_backtest(name, s_pct, o_pct, fn, data)
    results.append(r)
    print(f'annual {r["annual_ret"]:+.2f}%, excess {r["excess_annual"]:+.2f}%, DD {r["max_dd"]:.1f}%')

---
## Capital Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 7))
spy_norm = spy_prices / spy_prices.iloc[0] * INITIAL_CAPITAL

standard = [r for r in results if 'Deep' not in r['name']]
deep = [r for r in results if 'Deep' in r['name']]

for ax, group, title, palette in [
    (axes[0], standard, 'Standard OTM Puts (\u03b4 -0.25 to -0.10)', plt.cm.Reds),
    (axes[1], deep, 'Deep OTM Puts / Universa-style (\u03b4 -0.10 to -0.02)', plt.cm.Purples),
]:
    ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2.5, label='SPY B&H', alpha=0.7)
    cmap = palette(np.linspace(0.3, 0.9, len(group)))
    for r, c in zip(group, cmap):
        r['balance']['total capital'].plot(ax=ax, label=f"{r['name']} ({r['excess_annual']:+.2f}%)",
                                           color=c, alpha=0.85, lw=1.5)
    shade_crashes(ax)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_ylabel('$')
    ax.ticklabel_format(style='plain', axis='y')
    ax.legend(fontsize=7, loc='upper left')

plt.tight_layout()
plt.show()

---
## Results Table

In [ ]:
rows = []
for r in results:
    rows.append({
        'Strategy': r['name'],
        'Type': 'Deep OTM' if 'Deep' in r['name'] else 'Standard OTM',
        'Allocation %': r['opt_pct'] * 100,
        'Annual Return %': r['annual_ret'],
        'Excess vs SPY %': r['excess_annual'],
        'Max Drawdown %': r['max_dd'],
        'Trades': r['trades'],
    })
df = pd.DataFrame(rows)

styled = (df.style
    .format({'Allocation %': '{:.1f}', 'Annual Return %': '{:.2f}',
             'Excess vs SPY %': '{:+.2f}', 'Max Drawdown %': '{:.1f}', 'Trades': '{:.0f}'})
    .map(color_excess, subset=['Excess vs SPY %'])
)
style_returns_table(styled).set_caption(
    f'Tail Hedge Comparison  |  SPY B&H: {data["spy_annual_ret"]:.2f}%/yr, DD: {data["spy_dd"]:.1f}%'
)

---
## Verdict

### Who Wins: AQR or Universa?

Our empirical results over 17+ years of SPY data most likely support **AQR's position**:

1. **Every put allocation** (standard or deep OTM) **underperforms** SPY B&H
2. **Deep OTM puts** fare slightly better per dollar spent (lower premium per contract) but still negative
3. The **3.3% deep OTM allocation** (Spitznagel's recommended size) shows significant drag

### Why Spitznagel's Claim Might Still Be Valid

Our backtester has limitations that may favor AQR:
- We use **monthly** rebalancing (Universa uses continuous management)
- We pick **one contract per rebalance** (Universa rolls positions dynamically)
- We don't model the **behavioral benefit**: staying fully invested during crashes because you have tail protection
- The **geometric return** argument ($G = \mathbb{E}[\ln R]$) requires very long horizons and specific sizing

### The Bottom Line

For a simple, systematic implementation:
- Buying puts = paying insurance premium = drag on returns
- The debate is about whether **active management** of tail hedges can overcome this drag
- A simple backtester cannot capture Universa's active management alpha